In [1]:
import logging

logger = logging.getLogger(__name__)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models.callbacks import CallbackAny2Vec

class callback(CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        prev = 0
        loss = model.get_latest_training_loss()
        change_loss = loss - prev
        prev = loss
        print('Loss after epoch {}: {}'.format(self.epoch, change_loss))
        self.epoch += 1

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-05-28 20:51:26,667 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2024-05-28 20:51:26,668 : INFO : built Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...> from 9 documents (total 29 corpus positions)
2024-05-28 20:51:26,669 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...> from 9 documents (total 29 corpus positions)", 'datetime': '2024-05-28T20:51:26.669356', 'gensim': '4.3.2', 'python': '3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]', 'platform': 'Linux-5.15.146.1-microsoft-standard-WSL2-x86_64-with-glibc2.35', 'event': 'created'}


Define the corpus, cleaning for punctuation, tags and multiple whitespaces
Shuffe the corpus
Yield each sentence of the shuffled corpus

In [3]:
from gensim.test.utils import datapath
import gensim.parsing.preprocessing as gsp
import random

class EventCorpus:
	"""An iterator that yields sentences (lists of str) for pseudo event corpus."""
	def __iter__(self):
		sentences = []
		corpus_path = datapath('/home/vlnsha004/CSC2005Z/lpe_soccerevents/data/eventcorpus8.txt')
		
		for line in open(corpus_path):

			# remove all single quotation marks
			line = line.replace("'", "") # Erroneous quotation marks
			line = line.replace('6', '5')
			line = line.replace('"', '')
			CUSTOM_FILTERS = [gsp.strip_tags, gsp.strip_multiple_whitespaces]
			sentences.append(gsp.preprocess_string(line, CUSTOM_FILTERS))
			yield gsp.preprocess_string(line, CUSTOM_FILTERS)
        
		random.shuffle(sentences)  # Shuffle the sentences

		for sentence in sentences:
			yield sentence

Model training
Current hyperparameters:
window = 3
epochs = 35
vs = 500
mc = 1 # min_count
negative = 20, Negative used for the sample size used for negative sampling

In [4]:
from gensim.models import Word2Vec

# Train the model on the new data
# model.train(sequences, total_examples=model.corpus_count, epochs=model.epochs, compute_loss = True, callbacks=[callback()])

sequences = EventCorpus()

window = 10
epochs = 60
vs = 800
mc = 1 # min_count
ns = 20 # negative sampling
subsample = 1e-5
model = Word2Vec(sg = 1, hs =1, sentences=sequences, epochs = epochs, sample=subsample, vector_size=vs, window=window, min_count=mc, workers=10, compute_loss = True, negative=ns, callbacks=[callback()]) #Obtain params from football2vec text
model.save(f"/home/vlnsha004/CSC2005Z/lpe_soccerevents/models/anon/Seq2vec_{epochs}_{vs}_{window}_{mc}_{ns}_shuffled_sample_skip_hs.model") # First model trained on full corpus

2024-05-28 20:51:26,734 : INFO : collecting all words and their counts
2024-05-28 20:51:26,737 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-05-28 20:51:26,849 : INFO : PROGRESS: at sentence #10000, processed 58929 words, keeping 147 word types
2024-05-28 20:51:26,965 : INFO : PROGRESS: at sentence #20000, processed 117432 words, keeping 191 word types
2024-05-28 20:51:27,067 : INFO : PROGRESS: at sentence #30000, processed 175749 words, keeping 225 word types
2024-05-28 20:51:27,169 : INFO : PROGRESS: at sentence #40000, processed 234430 words, keeping 276 word types
2024-05-28 20:51:27,330 : INFO : PROGRESS: at sentence #50000, processed 293093 words, keeping 322 word types
2024-05-28 20:51:27,431 : INFO : PROGRESS: at sentence #60000, processed 351564 words, keeping 347 word types
2024-05-28 20:51:27,536 : INFO : PROGRESS: at sentence #70000, processed 410018 words, keeping 361 word types
2024-05-28 20:51:27,642 : INFO : PROGRESS: at sentence #80000

Loss after epoch 0: 2935273.0


2024-05-28 20:56:20,369 : INFO : EPOCH 1 - PROGRESS: at 0.28% examples, 20219 words/s, in_qsize 17, out_qsize 0
2024-05-28 20:56:21,371 : INFO : EPOCH 1 - PROGRESS: at 0.77% examples, 30622 words/s, in_qsize 18, out_qsize 1
2024-05-28 20:56:22,372 : INFO : EPOCH 1 - PROGRESS: at 1.20% examples, 30140 words/s, in_qsize 20, out_qsize 1
2024-05-28 20:56:23,390 : INFO : EPOCH 1 - PROGRESS: at 1.75% examples, 31657 words/s, in_qsize 17, out_qsize 1
2024-05-28 20:56:24,443 : INFO : EPOCH 1 - PROGRESS: at 2.25% examples, 31531 words/s, in_qsize 19, out_qsize 0
2024-05-28 20:56:25,452 : INFO : EPOCH 1 - PROGRESS: at 2.66% examples, 30875 words/s, in_qsize 20, out_qsize 0
2024-05-28 20:56:26,492 : INFO : EPOCH 1 - PROGRESS: at 3.17% examples, 31411 words/s, in_qsize 20, out_qsize 0
2024-05-28 20:56:27,518 : INFO : EPOCH 1 - PROGRESS: at 3.65% examples, 31946 words/s, in_qsize 20, out_qsize 1
2024-05-28 20:56:28,535 : INFO : EPOCH 1 - PROGRESS: at 4.18% examples, 33625 words/s, in_qsize 18, out_

Loss after epoch 1: 4710530.5


2024-05-28 20:59:37,988 : INFO : EPOCH 2 - PROGRESS: at 0.21% examples, 14462 words/s, in_qsize 19, out_qsize 0
2024-05-28 20:59:38,991 : INFO : EPOCH 2 - PROGRESS: at 0.62% examples, 23019 words/s, in_qsize 19, out_qsize 0
2024-05-28 20:59:39,994 : INFO : EPOCH 2 - PROGRESS: at 1.01% examples, 25415 words/s, in_qsize 17, out_qsize 0
2024-05-28 20:59:41,139 : INFO : EPOCH 2 - PROGRESS: at 1.38% examples, 24306 words/s, in_qsize 19, out_qsize 0
2024-05-28 20:59:42,226 : INFO : EPOCH 2 - PROGRESS: at 1.84% examples, 25149 words/s, in_qsize 20, out_qsize 1
2024-05-28 20:59:43,243 : INFO : EPOCH 2 - PROGRESS: at 2.34% examples, 26351 words/s, in_qsize 19, out_qsize 0
2024-05-28 20:59:44,268 : INFO : EPOCH 2 - PROGRESS: at 2.87% examples, 27588 words/s, in_qsize 19, out_qsize 0
2024-05-28 20:59:45,710 : INFO : EPOCH 2 - PROGRESS: at 3.26% examples, 26063 words/s, in_qsize 18, out_qsize 1
2024-05-28 20:59:46,731 : INFO : EPOCH 2 - PROGRESS: at 3.66% examples, 26619 words/s, in_qsize 20, out_

...

2024-05-29 00:26:49,882 : INFO : EPOCH 58 - PROGRESS: at 0.33% examples, 24601 words/s, in_qsize 0, out_qsize 0
2024-05-29 00:26:50,942 : INFO : EPOCH 58 - PROGRESS: at 0.64% examples, 23591 words/s, in_qsize 0, out_qsize 0
2024-05-29 00:26:52,011 : INFO : EPOCH 58 - PROGRESS: at 0.77% examples, 19486 words/s, in_qsize 0, out_qsize 0
2024-05-29 00:26:53,017 : INFO : EPOCH 58 - PROGRESS: at 1.04% examples, 19110 words/s, in_qsize 0, out_qsize 0
2024-05-29 00:26:54,032 : INFO : EPOCH 58 - PROGRESS: at 1.31% examples, 18954 words/s, in_qsize 0, out_qsize 0
2024-05-29 00:26:55,034 : INFO : EPOCH 58 - PROGRESS: at 1.58% examples, 18811 words/s, in_qsize 0, out_qsize 0
2024-05-29 00:26:56,070 : INFO : EPOCH 58 - PROGRESS: at 1.92% examples, 19131 words/s, in_qsize 0, out_qsize 0
2024-05-29 00:26:57,097 : INFO : EPOCH 58 - PROGRESS: at 2.22% examples, 19165 words/s, in_qsize 0, out_qsize 0
2024-05-29 00:26:58,109 : INFO : EPOCH 58 - PROGRESS: at 2.51% examples, 19252 words/s, in_qsize 0, out_

Loss after epoch 58: 53197436.0


2024-05-29 00:31:26,364 : INFO : EPOCH 59 - PROGRESS: at 0.32% examples, 23756 words/s, in_qsize 0, out_qsize 1
2024-05-29 00:31:27,397 : INFO : EPOCH 59 - PROGRESS: at 0.63% examples, 23581 words/s, in_qsize 1, out_qsize 0
2024-05-29 00:31:28,450 : INFO : EPOCH 59 - PROGRESS: at 0.95% examples, 23724 words/s, in_qsize 0, out_qsize 2
2024-05-29 00:31:29,450 : INFO : EPOCH 59 - PROGRESS: at 1.29% examples, 23558 words/s, in_qsize 0, out_qsize 0
2024-05-29 00:31:30,670 : INFO : EPOCH 59 - PROGRESS: at 1.51% examples, 20963 words/s, in_qsize 14, out_qsize 0
2024-05-29 00:31:31,749 : INFO : EPOCH 59 - PROGRESS: at 1.85% examples, 20934 words/s, in_qsize 18, out_qsize 1
2024-05-29 00:31:32,764 : INFO : EPOCH 59 - PROGRESS: at 2.22% examples, 21304 words/s, in_qsize 17, out_qsize 1
2024-05-29 00:31:33,868 : INFO : EPOCH 59 - PROGRESS: at 2.59% examples, 21650 words/s, in_qsize 19, out_qsize 0
2024-05-29 00:31:34,882 : INFO : EPOCH 59 - PROGRESS: at 2.92% examples, 21736 words/s, in_qsize 20,

Loss after epoch 59: 54273168.0


2024-05-29 00:35:59,832 : INFO : saved /home/vlnsha004/CSC2005Z/lpe_soccerevents/models/anon/Seq2vec_60_800_10_1_20_shuffled_sample_skip_hs.model
